# unsloth 微调

Unsloth 是目前最适合在消费级 GPU 上使用的微调框架，它的显存消耗极少且速度极快。最感人的是它的文档也是最全的，回答了初学者的常见疑惑和一些极有价值的问题 --> [⭐ Beginner? Start here!](https://docs.unsloth.ai/get-started/beginner-start-here)

In [1]:
# !uv pip install unsloth

In [14]:
import torch

from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer, AutoTokenizer, BitsAndBytesConfig
from unsloth import is_bfloat16_supported
from peft import AutoPeftModelForCausalLM

DATASET_PATH = './data/train_zh_1000.json'
MODEL_PATH = './model/Qwen/Qwen2.5-0.5B-instruct'     
OUTPUT_PATH = './model/lora_model'

## 1. 加载模型

In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = MODEL_PATH,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


## 2. 加载数据

与上一节相同，我们要加载 Alpaca 数据集，然后使用模板，将数据做成特定格式的纯文本。

官方Notebook [Qwen2.5_(7B)-Alpaca.ipynb](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_(7B)-Alpaca.ipynb) 提供的模板：

```
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
```

由于我们数据的 `Input` 字段为空，可以略去。我们使用的格式如下：

In [5]:
alpaca_prompt = """### Question:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [6]:
# 从 HuggingFace 仓库加载数据集
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")

# 从本地加载数据集
dataset = load_dataset("json", data_files=DATASET_PATH, split = "train")

# 打印数据集的基本信息
print(f'基本信息：\n{dataset}')

基本信息：
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1000
})


In [7]:
# 将模板应用于数据
dataset = dataset.map(formatting_prompts_func, batched = True,)

dataset[0]

{'instruction': '血热的临床表现是什么?',
 'input': '',
 'output': '初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。',
 'text': '### Question:\n血热的临床表现是什么?\n\n### Response:\n初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。<|im_end|>'}

## 3. 模型微调

In [8]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss
1,3.361200
2,3.536300
3,4.094300
4,4.205400
5,3.495500
6,3.208000
7,3.200600
8,3.059800
9,2.882400
10,3.060800


Unsloth: Will smartly offload gradients to save VRAM!


## 4. 模型推理

In [10]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "血热的临床症状是什么？", # instruction
        "", # input - leave this blank for generation!
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['### Question:\n血热的临床症状是什么？\n\n### Response:\n舌红苔黄或赤，脉细数。<|im_end|>']

In [11]:
# 实时推理
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### Question:
血热的临床症状是什么？

### Response:
面红；心烦；口渴；盗汗；手足心热；尿黄；烦躁；舌质红；脉细数；舌边尖红。<|im_end|>


## 5. 保存模型

In [12]:
# 保存模型
model.save_pretrained(OUTPUT_PATH)  # Local saving
tokenizer.save_pretrained(OUTPUT_PATH)
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('./model/lora_model/tokenizer_config.json',
 './model/lora_model/special_tokens_map.json',
 './model/lora_model/vocab.json',
 './model/lora_model/merges.txt',
 './model/lora_model/added_tokens.json',
 './model/lora_model/tokenizer.json')

## 6. 保存模型后重新加载

**1）使用 unsloth 加载模型**

In [13]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = OUTPUT_PATH, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
_ = FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [14]:
alpaca_prompt = """### Question:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "血热的临床症状是什么？", # instruction
        "", # input - leave this blank for generation!
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### Question:
血热的临床症状是什么？

### Response:
皮肤干燥；口干；舌红少津；心烦失眠；多食易饥；大便秘结；尿黄；小便赤热；眼花。<|im_end|>


**2）使用 transformers 加载模型**

In [15]:
# I highly do NOT suggest - use Unsloth if possible

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # 计算时使用的数据类型
    bnb_4bit_quant_type="nf4",             # 量化类型，nf4或fp4
    bnb_4bit_use_double_quant=True         # 是否使用双重量化
)

model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_PATH, # YOUR MODEL YOU USED FOR TRAINING
    quantization_config=quantization_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_PATH)

In [19]:
alpaca_prompt = """### Question:
{}

### Response:
{}"""

prompt = alpaca_prompt.format("血热的临床症状是什么？", "")

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id
)
# outputs

In [21]:
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

### Question:
血热的临床症状是什么？

### Response:
口干；心烦；尿黄；眩晕；消瘦；发热；便秘；烦躁；舌红少津。


参考：

- GitHub: [unslothai/unsloth](https://github.com/unslothai/unsloth)
- Docs: [docs.unsloth.ai](https://docs.unsloth.ai/)
- Notebook：[Qwen2.5_(7B)-Alpaca.ipynb](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_(7B)-Alpaca.ipynb)